In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\DataScienceProjects\\mobile-price-range-prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    metric_file_name: Path

In [5]:
from mobilePriceRangePrediction.constants import *
from mobilePriceRangePrediction.utils.common import read_yaml, create_directories, load_object, save_object

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config

In [6]:
from mobilePriceRangePrediction.utils.common import save_object ,load_object
from sklearn.metrics import f1_score, recall_score, precision_score
import pandas as pd


In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_model(self):
        # Get test data
        test_arr = load_object(self.config.data_path+'/test_arr.pkl')
        X_test, y_test = test_arr[:,:-1], test_arr[:,-1]

        # Get the model
        model = load_object(self.config.model_path+'/model.pkl')
        y_pred = model.predict(X_test)
        
        # Evaluate model
        f1 = f1_score(y_test, y_pred, average='micro')
        precision = precision_score(y_test, y_pred, average='micro')
        recall = recall_score(y_test, y_pred, average='micro')
        
        # Save as csv
        metrics_df = pd.DataFrame({
            'micro_F1_Score': [f1],
            'micro_Precision': [precision],
            'micro_Recall': [recall]
        })
        metrics_df.to_csv(self.config.metric_file_name, index=False)  

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_trainer_config = ModelEvaluation(config=model_evaluation_config)
    model_trainer_config.evaluate_model()
except Exception as e:
    raise e

[2024-03-26 18:12:27,328: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-26 18:12:27,330: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-26 18:12:27,331: INFO: common: Created directory at: artifacts]
[2024-03-26 18:12:27,332: INFO: common: Created directory at: artifacts/model]
